# Chainer 初心者ハンズオン
# はじめての画像分類

<hr>

## ページ内目次

<ul>
<li><a href="#画像ファイルの読み込み">画像ファイルの読み込み</a></li>
<li><a href="#Chainerのインポート、VGG16モデルのロード">Chainerのインポート、VGG16モデルのロード</a></li>
<li><a href="#モデルを使って画像のクラスを予測する">モデルを使って画像のクラスを予測する</a></li><li><a href="#Variableクラス中の数値データへのアクセス">Variableクラス中の数値データへのアクセス</a></li>
<li><a href="#複数画像をバッチ処理する">複数画像をバッチ処理する</a></li>
<li><a href="#model.predict()-が返す行列を理解する">model.predict()が返す行列を理解する</a></li>
<li><a href="#model.predict()-が返した確率をグラフ表示する">model.predict()が返した確率をグラフ表示する</a></li>
<li><a href="#ラベルリストの利用">ラベルリストの利用</a></li>
<li><a href="#Top-N-クラスの調査">Top-N-クラスの調査</a></li>
<li><a href="#Top-N-クラスの調査（クラスラベルとあわせて）">Top-Nクラスの調査（クラスラベルとあわせて）</a></li>
<li><a href="#matplotlib-による円グラフの表示">matplotlibによる円グラフの表示</a></li>
<li><a href="#さらなる演習">さらなる演習</a></li>
<li><a href="#様々な画像をまとめてロードする">様々な画像をまとめてロードする</a></li>
</ul>

<hr>

PyTorch 初心者ハンズオンにようこそ！

本ハンズオンでは、ディープラーニングやPyTorch フレームワークを利用したことがない方を対象として、ディープラーニングによる画像分類の方法をご紹介します。

<hr>

画像に何が映っているか、深層学習によって推測させてみましょう。PyTorchを使った場合、有名なネットワーク構造がいくつか、すぐに試せる状態になっています。ここでは、2015年に発表された VGG16 を使ってみましょう。

<div style="border: 1px solid; padding: 10px">
VGG16ネットワークは、224x224の画像を1000クラスに分類するためのネットワークです。

<img src="_images/vgg16.png">
図: VGG16ネットワークの構造
<p>
Very Deep Convolutional Networks for Large-Scale Image Recognition<br>https://arxiv.org/abs/1409.1556

<p>Chainer では links.model.vision.vgg.VGG16Layers クラスが定義されています。 Chainer 自体に学習済みデータは提供されていませんが、 Caffe 向けに公開されている学習済みモデルを読み込んで利用できるため、学習に時間をかけなくても試せます。

https://github.com/chainer/chainer/blob/v2/chainer/links/model/vision/vgg.py

</div>

この環境には<a href="00_sample_images.ipynb">以下の画像</a>が準備されています。

<table><tr style="align: center"><td>airplane.jpg</td><td>cat.jpg</td><td>dog.jpg</td><td>dolphin.jpg</td><td>human_1.jpg</td><td>human_2.jpg</td><td>spider.jpg</td></tr>
<tr><td><img src="airplane.jpg" style="width: 96px"></td><td><img src="cat.jpg" style="width: 96px"></td><td><img src="dog.jpg"  style="width: 96px"></td><td><img src="dolphin.jpg" style="width: 96px"></td><td><img src="human_1.jpg" style="width: 96px"></td><td><img src="human_2.jpg" style="width: 96px"></td><td><img src="spider.jpg" style="width: 96px"></td></tr></table>

これらの画像から1点選択し、読み込みます。下記の Notebook は cat.jpg を選択した選定で解説します。

<hr>

## 画像ファイルの読み込み

In [ ]:
from google.colab import drive
drive_path = '/content/drive/'
drive.mount(drive_path)
pj_path = drive_path + 'My Drive/ColabNotebooks/PyTorchBeginner/'

In [ ]:
import numpy as np

In [ ]:
from PIL import Image

img = Image.open(pj_path + "_images/cat.jpg")

In [ ]:
img

上記ブロックを実行し、猫が表示されましたか？

一応、サイズを確認しておきましょう。

In [ ]:
img.size

<hr>

## PyTorchのインポート、VGG16モデルのロード

PyTorchにはDataLoaderというモジュールが存在する。

- torchvision
- torchtext
- torchaudio

PyTorchのインポート、訓練済みモデルのロード

In [ ]:
import torch

In [ ]:
from torchvision import datasets, models, transforms

In [ ]:
model = models.vgg16_bn(pretrained=True)

ここで、 model には、 VGG16 と呼ばれる有名な画像分類ネットワークがロードされます。

<div style="border: 1px solid; padding: 10px">
<tt>In[*]</tt> という表示がしばらく続くかもしれません。VGG16のデータファイルがない場合、インターネットからデータ(約500MB)をダウンロードし変換する必要があるため、辛抱強く待ってみてください。</div>

機械学習において、初期値はランダムな値が用いられる。しかし、それでは結果が変わってしまって初心者の学習を阻害してしまうため、乱数を固定する必要がある。  
PyTorchのモデルはeval()で評価モードに切り替えることができる。

In [ ]:
model.eval()

モデルの確認

In [ ]:
type(model)

model オブジェクトが torchvision.models.vgg.VGG クラスであることを確認してみてください。


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = model.to(device)

<hr>

## 前処理を行う


PyTorchでは、画像をテンソル型にする必要がある。  
前処理をまとめて定義しておく。

In [ ]:
preprocess = transforms.Compose([
                                  transforms.Resize(256),
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

前処理を実行して、画像からテンソルを取得してみる。

In [ ]:
img_tensor = preprocess(img).cuda()
img_tensor.shape

In [ ]:
type(img_tensor)

tensorの順番は少し直観的ではないかもしれない。  
CH, Height, Widthの順番に格納されているので、さっき確認した画像サイズと照らし合わせてみよう。


画像をモデルに入力するときは、3次元テンソルではなく、４次元テンソルである必要がある。  
何の要素が必要かというと、入力する画像の枚数データである。  
これを**バッチサイズ**と呼ぶため、必ず覚えておいてほしい。

今回は１枚だけを入力するので、0次元目に１を新規要素として追加する。


In [ ]:
img_tensor.unsqueeze_(0)
img_tensor.size()

<hr>

## モデルを使って画像のクラスを予測する

テンソルをニューラルネットワークに入力し、出力 p を得る

モデルにテンソルを渡す場合はVariableにする必要がある。  
Variableクラスをインポートしよう。

In [ ]:
from torch.autograd import Variable

In [ ]:
p = model(Variable(img_tensor))

※ model.predict() の実行には、いくらか時間がかかります。



予測結果（の生データ）を表示してみる

In [ ]:
p

たくさんの値がはいった行列（Variable型）が見えるはずです。

<hr>

## Variableクラス中の数値データへのアクセス

Variable クラスはPyTorchが行列を扱うための構造です。中に入っている行列に数値としてアクセスしたい場合は、 p.data を試してみてください。

In [ ]:
p.__class__

In [ ]:
p.data.__class__

In [ ]:
p[0, 1]

In [ ]:
p.data[0, 1]

<hr>

## 複数画像をバッチ処理する

複数の画像を連続で処理したい場合は、配列として画像を複数渡して、まとめて予測を得ることができます。この際、 modelが返すテンソル（イメージ数、 1000クラス）になります。

1枚単位で画像を予測する場合と結果は一緒ですが、GPU搭載サーバーで短時間に多数の画像を予測したい場合、多数の画像をまとめることにより、システム全体のスループットが向上します。


### 1枚のみ処理する場合

In [ ]:
img_tensor1 = preprocess(img)
img_tensor1.unsqueeze_(0)
img_tensor.shape

In [ ]:
p1 =  model(Variable(img_tensor1).cuda())

In [ ]:
p1.shape # => (1, 000)

### 2枚処理する場合

操作は基本的にNumpyに準拠しているため、それと同じように次元を操作する。  
今回は、新しい次元を軸にテンソルを連結したいため、stack関数を用いる。  
まず、前に解説したSeriesの次元が2になっていることを確認しよう。

In [ ]:
img_tensor2 = preprocess(img).cuda()
img_tensor2 = torch.stack([img_tensor2, img_tensor2])
img_tensor2.shape

In [ ]:
p2 =  model(Variable(img_tensor2).cuda())

In [ ]:
p2.shape # 演習: 結果を予想してください

### 3枚処理する場合

【演習】<br><br>
３枚の画像をmodelに渡して、期待どおりのテンソルが戻されることを確認してください。

In [ ]:
img_tensor3 = preprocess(img).cuda()
img_tensor3 = torch.stack([img_tensor3, img_tensor3, img_tensor3])
p3 = model(Variable(img_tensor3).cuda())
p3.shape

<hr>

## modelが返した確率をグラフ表示する


以降、予測した最初の画像 (0行目) の内容について調査していきます。  
変数 d に p.data[0] を代入し、以降の操作は d に対して行います。


In [ ]:
d = p.data[0].to("cpu")

In [ ]:
d.shape # dの要素数を確認したい方はお試しください

In [ ]:
d # 中身を見たい方はお試しください

これからこのデータをグラフに直していきますが、マイナス値が含まれているためそれらを修正します。  
1000項目の内どの程度その項目らしいかを表しているので、これを確率値に直してみます。

> 厳密には確率値ではないです。

In [ ]:
import torch.nn.functional as F

In [ ]:
softmax_d = F.softmax(d, dim=0)
softmax_d[:5]

d を棒グラフとして表示してみます。

In [ ]:
import matplotlib.pyplot as plt
 
%matplotlib inline 
plt.style.use('ggplot')
plt.figure(figsize=(20,10))
plt.bar(x=range(len(softmax_d)), height=softmax_d)
plt.plot()

<div style="border: 1px solid; padding: 10px">これは、0番目から999番目のクラスまで、そのクラスである確率の棒グラフです。200 ~ 400 の間にもっとも高いバーが見えているはずですが、この最も高い値がいくつめの要素か調べていきます。</div>

配列 d の中から最大値を探す


In [ ]:
predicted_label = np.argmax(d.numpy())

最大値がある要素の番号を表示


In [ ]:
predicted_label

# d[predicted_label] は d の中の最大値で、確率（0.0〜1.0）


In [ ]:
d[predicted_label]

`np.argmax(d.numpy())` が 281  を返した場合、入力した画像 img は 281 番目のクラスに分類されたことを意味しています。

<hr>

## ラベルリストの利用

入力した画像は、 predicted_label 番目（285番目）のクラスである確率が高いことがわかりました。しかし、この 285 番目のクラスというのは何を指しているのでしょうか。

（このモデルの学習時に利用された）クラスラベルリスト <a href="synset_words.txt">synset_words.txt</a>  と照合することで、このクラスラベルが何を意味しているのかを調べます。

In [ ]:
labels = open(pj_path+"synset_words.txt", "r").readlines()

labels の内容を確認する (1000個まで表示すると長いので、とりあえず最初の10個まで)

In [ ]:
labels[:10]

入力画像に対して最も高い確率を示したクラスのラベル文字列を表示してみよう。  
predict_label 番目の要素を調べる


In [ ]:
labels[predicted_label]

実行した結果、tabby, tabby cat、つまり、ニューラルネットワークは、この画像が  tabby, tabby catに分類した、ということになります。

> 別のライブラリのChainerで同じモデルを使ってやってみたらEgyptian catになりました。

<hr>

## Top-N クラスの調査

<div style="border: 1px solid; padding: 10px">返された値は「入力された画像が各クラスの値である確率」だと説明しました。また、最大値は 285番目の要素であることがわかりました。最大値に続く、大きな値を探してみましょう。</div>

確率を昇順にソートして、最後の最大3件を表示


In [ ]:
d.topk(3)

<hr>

## Top-N クラスの調査（クラスラベルとあわせて）

<div style="border: 1px solid; padding: 10px">このままではクラスラベルがなく、確率値が何に対応しているのかがわかりません。以下の操作で確率とクラスラベルをマージした上でソートしてみましょう。</div>

確率とクラスラベルを組みわせたリストの作成

In [ ]:
p2 = list(zip(softmax_d.numpy(), labels))
p2[:10]

作成した組み合わせリストから Top 3 を表示

`topk`を使わなくても、リスト形式の演算もできます。

In [ ]:
sorted(p2, reverse=True)[:3]

<div style="border: 1px solid; padding: 10px">
出力例

<p>
[<br>(0.52825135, 'n02124075 Egyptian cat\n'), <br>
   (0.11501167, 'n02123045 tabby, tabby cat\n'), <br>
   (0.052949127, 'n02123159 tiger cat\n')<br>]
</div>

<hr>

【演習】<br><br>同様に、Top-10 クラスを確認してください。

In [ ]:
p2 = list(zip(softmax_d.to("cpu").numpy(), labels))
sorted(p2, reverse=True)[:10]

<hr>

## matplotlib による円グラフの表示

<div style="border: 1px solid; padding: 10px">降順にソートした確率一覧を matplotlib の円グラフにすることで、Top-Nクラスがどれぐらいの割合を占めるのかを視覚的に確認できます。</div>

In [ ]:
softmax_d_sorted = sorted(softmax_d, key=lambda x:-x)
softmax_d_sorted[:10]

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'font.size':15})
plt.figure(figsize=(10,10))
plt.pie(softmax_d_sorted)
plt.plot()

<hr>

## さらなる演習

【演習】<br>

* ハンズオン環境に用意されている画像を model にわたし、各画像の予測クラス（１画像あたり1クラス）を表示してください。


<a href="00_sample_images.ipynb">ハンズオン環境に存在する画像</a>をまとめて読み込みたい場合は、次のブロックを実行してください。

In [ ]:
img_airplane = Image.open(pj_path+"_images/airplane.jpg")
img_cat = Image.open(pj_path+"_images/cat.jpg")
img_dog = Image.open(pj_path+"_images/dog.jpg")
img_dolphin = Image.open(pj_path+"_images/dolphin.jpg")
img_human1 = Image.open(pj_path+"_images/human_1.jpg")
img_human2 = Image.open(pj_path+"_images/human_2.jpg")
img_spider = Image.open(pj_path+"_images/spider.jpg")

In [ ]:
# 演習
img_tensor_ = preprocess(img_airplane).cuda()
img_tensor_.unsqueeze_(0)
p_ = model(img_tensor_)